In [1]:
%matplotlib inline
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

In [2]:
F.one_hot(torch.tensor([0, 2]), len(vocab))

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])

In [3]:
X = torch.arange(10).reshape((2, 5))
F.one_hot(X.T, 28).shape

torch.Size([5, 2, 28])

In [4]:
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01

    # 隐藏层参数
    W_xh = normal((num_inputs, num_hiddens))
    W_hh = normal((num_hiddens, num_hiddens))
    b_h = torch.zeros(num_hiddens, device=device)
    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)
    # 附加梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

In [5]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [6]:
def rnn(inputs, state, params):
    # inputs的形状：(时间步数量，批量大小，词表大小)
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    # X的形状：(批量大小，词表大小)
    for X in inputs:
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)
        Y = torch.mm(H, W_hq) + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H,)

In [7]:
class RNNModelScratch: #@save
    """从零开始实现的循环神经网络模型"""
    def __init__(self, vocab_size, num_hiddens, device,
                 get_params, init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn

    def __call__(self, X, state):
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params)

    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

In [10]:
num_hiddens = 512
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params,
                      init_rnn_state, rnn)
state = net.begin_state(X.shape[0], d2l.try_gpu())
Y, new_state = net(X.to(d2l.try_gpu()), state)
# 这里直接打印的X还没有经过one-hot
Y.shape, len(new_state), new_state[0].shape,Y,X

(torch.Size([10, 28]),
 1,
 torch.Size([2, 512]),
 tensor([[ 8.5908e-05, -4.1410e-03, -1.2673e-03, -8.6080e-05,  3.2799e-03,
          -1.2756e-03,  1.1435e-03,  7.9507e-04, -3.5946e-03,  2.0983e-03,
          -1.2371e-03,  3.8638e-04, -4.3222e-03, -4.8007e-04,  2.4422e-03,
           1.2990e-03, -1.8758e-03,  6.4456e-04, -4.4322e-04, -2.0868e-03,
          -1.2760e-03, -1.3711e-03,  1.1774e-03, -1.1880e-03,  1.4271e-03,
           2.0547e-03, -1.7830e-04,  5.4665e-04],
         [-6.4587e-04, -5.5983e-04, -9.5121e-04, -5.3750e-04, -3.3886e-03,
          -1.2566e-03, -3.3099e-03,  3.0064e-04,  2.4458e-03,  1.5325e-04,
          -1.1496e-04, -4.4658e-05,  6.7247e-04, -4.0525e-04,  8.9456e-04,
          -2.0999e-03,  3.0081e-03, -1.2796e-03, -7.7973e-04,  3.3528e-04,
           1.6639e-03,  5.2455e-03,  8.9568e-04,  3.1508e-03,  7.5633e-04,
          -5.2787e-04, -2.3636e-03, -5.6478e-04],
         [ 1.3061e-03,  5.3973e-04,  6.9481e-04, -1.1925e-03, -4.2591e-03,
          -7.1724e-04, -1